# Update Completion Status - tsDTA Processor

**Purpose**: Update parent document status to READY_FOR_VERSIONING after all processing is complete.

**Input**: Task values from upstream tasks (extract_excel_sheets, load_codelist, load_transfer_metadata)

**Output**: Updated md_dta_history.current_status for processed parent documents

**Logic**:
1. Read status from upstream tasks
2. If all required documents processed, mark parent as READY_FOR_VERSIONING
3. Update parent document status in md_dta_history


In [ ]:
# Cell 0: Imports
import json
from datetime import datetime
from pyspark.sql import functions as F
from pyspark.sql.functions import lit, current_timestamp
from delta.tables import DeltaTable


In [ ]:
# Cell 1: Get Configuration from Setup Task
globals_dict = json.loads(dbutils.jobs.taskValues.get(taskKey="setup", key="globals"))
streaming_config = json.loads(dbutils.jobs.taskValues.get(taskKey="setup", key="streaming_config"))
created_by_principal = dbutils.jobs.taskValues.get(taskKey="setup", key="created_by_principal")
databricks_job_id = dbutils.jobs.taskValues.get(taskKey="setup", key="databricks_job_id")
databricks_run_id = dbutils.jobs.taskValues.get(taskKey="setup", key="databricks_run_id")

catalog = globals_dict['catalog']
bronze_schema = globals_dict['bronze_schema']

# Get required tags for versioning from config
required_tags = streaming_config.get('completion_tracking', {}).get('required_tags_for_versioning', ['tsDTA'])

history_table = f"{catalog}.{bronze_schema}.md_dta_history"

print(f"Catalog: {catalog}")
print(f"History table: {history_table}")
print(f"Required tags for versioning: {required_tags}")


In [ ]:
# Cell 2: Check Upstream Task Status
print(f"{'='*80}")
print(f"Checking upstream task status...")
print(f"{'='*80}")

# Check extract_excel_sheets status
try:
    excel_status = dbutils.jobs.taskValues.get(taskKey="extract_excel_sheets", key="excel_extraction_status")
    excel_success_count = int(dbutils.jobs.taskValues.get(taskKey="extract_excel_sheets", key="excel_success_count"))
    excel_error_count = int(dbutils.jobs.taskValues.get(taskKey="extract_excel_sheets", key="excel_error_count"))
    print(f"  extract_excel_sheets: {excel_status}")
    print(f"    SUCCESS: {excel_success_count}, ERROR: {excel_error_count}")
except Exception as e:
    print(f"  extract_excel_sheets: Could not read status - {e}")
    excel_status = "UNKNOWN"
    excel_success_count = 0

# Check load_codelist status
try:
    codelist_status = dbutils.jobs.taskValues.get(taskKey="load_codelist", key="codelist_status")
    codelist_success = int(dbutils.jobs.taskValues.get(taskKey="load_codelist", key="codelist_success_count"))
    print(f"  load_codelist: {codelist_status} ({codelist_success} records)")
except Exception as e:
    print(f"  load_codelist: Could not read status - {e}")
    codelist_status = "UNKNOWN"

# Check load_transfer_metadata status
try:
    transfer_status = dbutils.jobs.taskValues.get(taskKey="load_transfer_metadata", key="transfer_variables_status")
    transfer_success = int(dbutils.jobs.taskValues.get(taskKey="load_transfer_metadata", key="transfer_variables_success_count"))
    transfer_error = int(dbutils.jobs.taskValues.get(taskKey="load_transfer_metadata", key="transfer_variables_error_count"))
    print(f"  load_transfer_metadata: {transfer_status}")
    print(f"    SUCCESS: {transfer_success}, ERROR: {transfer_error}")
except Exception as e:
    print(f"  load_transfer_metadata: Could not read status - {e}")
    transfer_status = "UNKNOWN"

print(f"{'='*80}")


In [ ]:
# Cell 3: Update Parent Document Status
# Mark parent documents as READY_FOR_VERSIONING if processing succeeded

print(f"{'='*80}")
print(f"Updating parent document status...")
print(f"{'='*80}")

# Determine new status based on upstream task results
if excel_status == "SUCCESS" and transfer_status in ["SUCCESS", "MANUAL_REVIEW_REQUIRED"]:
    # Processing succeeded (even if some records need review)
    new_status = "READY_FOR_VERSIONING"
elif excel_status == "FAILED" or transfer_status == "FAILED":
    new_status = "PROCESSING_FAILED"
else:
    new_status = "PROCESSING_INCOMPLETE"

print(f"New status to set: {new_status}")

# Find parent documents that were processed in this run
# Use the documents that had status = EXCEL_EXTRACTION_COMPLETED
df_processed = spark.table(history_table).filter(
    (F.col("current_status") == "EXCEL_EXTRACTION_COMPLETED") &
    (F.col("databricks_run_id") == databricks_run_id) &
    (F.col("is_current") == True)
).select("document_id")

processed_count = df_processed.count()
print(f"Documents to update: {processed_count}")

if processed_count > 0:
    # Update status using MERGE
    delta_table = DeltaTable.forName(spark, history_table)
    
    delta_table.alias("target").merge(
        df_processed.alias("updates"),
        "target.document_id = updates.document_id"
    ).whenMatchedUpdate(set={
        "current_status": lit(new_status),
        "current_status_timestamp": current_timestamp(),
        "last_updated_by_principal": lit(created_by_principal),
        "last_updated_ts": current_timestamp(),
        "databricks_job_id": lit(databricks_job_id),
        "databricks_run_id": lit(databricks_run_id)
    }).execute()
    
    print(f"✓ Updated {processed_count} documents to status: {new_status}")
else:
    print("No documents to update.")

# Set task values for downstream tasks
dbutils.jobs.taskValues.set(key="completion_status", value=new_status)
dbutils.jobs.taskValues.set(key="documents_ready_count", value=str(processed_count))

print(f"\n✓ Completion tracking complete")
print(f"{'='*80}")
